# Load Data with Vectors

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

print(WEAVIATE_URL)
print(WEAVIATE_KEY)

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),
)

client.is_ready()

In [ ]:
from weaviate.classes.config import Configure

def create_wiki_collection():
    if client.collections.exists("Wiki"):
        client.collections.delete("Wiki")

    # Create a collection here - with Weaviate vectorizer and define source properties
    client.collections.create(
        name="Wiki",

        vectorizer_config=[
            Configure.NamedVectors.text2vec_weaviate(
                name="main_vector",

                # TODO: use model Snowflake/snowflake-arctic-embed-l-v2.0
                # TODO: set source properties to title and text
                
            )
        ],
    )

create_wiki_collection()

## Load the data from parquet files

In [1]:
from datasets import load_dataset

def prepare_dataset():
    return load_dataset('parquet', data_files={'train': ['../wiki-data/weaviate/snowflake-arctic-v2/*.parquet']}, split="train", streaming=True)
    # return load_dataset("weaviate/wiki-sample", "weaviate-snowflake-arctic-v2", split="train", streaming=True)

### Parquet Test
The parquet files should be located in "datasets/openai".

In [ ]:
dataset = prepare_dataset()

counter = 10
for i in dataset:
    print(i)

    counter -= 1
    if(counter == 0): break

### The import function

`TODO:`
* add a function to add objects to batch

In [ ]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

def import_wiki_data(max_rows=10_000):
    print(f"Importing {max_rows} data items")

    dataset = prepare_dataset()
    wiki = client.collections.get("Wiki")

    counter = 0

    with wiki.batch.fixed_size(batch_size=2000, concurrent_requests=2) as batch:
        for item in tqdm(dataset, total=max_rows):

            data_to_insert = {   
                "wiki_id": item["wiki_id"],
                "text": item["text"],
                "title": item["title"],
                "url": item["url"],
            }

            item_id = generate_uuid5(item["wiki_id"])

            item_vector = {
                "main_vector": item["vector"]
            }

            # TODO: add objects to batch using
            batch.add_object(
                # * data_to_insert
                # * item_id
                # * item_vector
            )

            # Check number of errors while running
            if(batch.number_errors > 10):
                print(f"Reached {batch.number_errors} errors during batch import")
                break
            
            # stop after the request number reaches = max_rows
            counter += 1
            if counter >= max_rows:
                break
    
    # check for errors at the end
    if (len(wiki.batch.failed_objects)>0):
        print("Final error check")
        print(f"Some errors {len(wiki.batch.failed_objects)}")
        print(wiki.batch.failed_objects[-1])
    
    print(f"Imported {counter} items")
    print("-----------------------------------")

In [ ]:
import_wiki_data(10_000)

## Check if data loaded correctly

In [ ]:
wiki = client.collections.get("Wiki")
len(wiki)

In [ ]:
res = wiki.query.fetch_objects(limit=1, include_vector=True)
print(res.objects[0].properties)
print(res.objects[0].vector)

In [ ]:
client.close()